In [22]:
import cv2
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import BinaryType

In [4]:
# %pip install numpy
# %pip install opencv-python

In [25]:
# Initialize SparkSession
spark = SparkSession.builder.appName("ReadVideoFromHDFS").getOrCreate()
spark

In [13]:
# Define a UDF to read video file
def read_video_file(path):
    cap = cv2.VideoCapture(path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for i in range(frame_count):
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame.tobytes())
    cap.release()
    return frames

# Register UDF with Spark
read_video_file_udf = udf(read_video_file, BinaryType())

In [19]:
# Read video file and convert to PySpark DataFrame
df = spark.read.format("binaryFile").load("hdfs://localhost:9870/kafka_output/video_test.mp4").selectExpr("path", "read_video_file(content) as video_data")

24/02/13 19:23:01 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: hdfs://localhost:9870/kafka_output/video_test.mp4.
org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1936)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


Py4JJavaError: An error occurred while calling o78.load.
: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1936)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


In [26]:
# Read video file from HDFS as binary data
video_df = spark.read.format("binaryFile").option("pathGlobFilter", "*.mp4").load("hdfs://localhost:9870/kafka_output/video_test.mp4")

# Show DataFrame schema and first few rows
video_df.printSchema()
video_df.show(truncate=False)

24/02/13 20:25:59 WARN FileStreamSink: Assume no metadata directory. Error while looking for metadata directory in the path: hdfs://localhost:9870/kafka_output/video_test.mp4.
org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1936)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


Py4JJavaError: An error occurred while calling o98.load.
: org.apache.hadoop.ipc.RpcException: RPC response exceeds maximum data length
	at org.apache.hadoop.ipc.Client$IpcStreams.readResponse(Client.java:1936)
	at org.apache.hadoop.ipc.Client$Connection.receiveRpcResponse(Client.java:1238)
	at org.apache.hadoop.ipc.Client$Connection.run(Client.java:1134)


In [4]:
# Check if the video opened successfully
if not cap.isOpened():
    print("Error: Could not open the video.")
    exit()

# Read and display each frame until the video ends
while cap.isOpened():
    ret, frame = cap.read()  # Read a frame
    if not ret:
        break  # Break the loop if there are no more frames
    cv2.imshow("Video", frame)  # Display the frame
    if cv2.waitKey(25) & 0xFF == ord("q"):
        break  # Press 'q' to exit

# Release the VideoCapture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

Error: Could not open the video.
